# importing requred libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
data_fake = pd.read_csv("Fake.csv")
data_true = pd.read_csv("True.csv")

# Loading data sets and preview data

In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


#  Loading Fake and True News Datasets

In [5]:
data_fake["class"] = 0
data_true["class"] = 1

# shape and index reset

In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake.reset_index(drop=True, inplace=True)
data_true.reset_index(drop=True, inplace=True)

# Separating 10 records for manual testing from each data set

In [8]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(len(data_fake)-1, len(data_fake)-11, -1):
    data_fake.drop([i], axis=0, inplace=True)

data_true_manual_testing = data_true.tail(10)
for i in range(len(data_true)-1, len(data_true)-11, -1):
    data_true.drop([i], axis=0, inplace=True)

# preparing manual test data

In [17]:
data_fake_manual_testing = data_fake_manual_testing.copy()
data_fake_manual_testing['class'] = 0

In [19]:
data_true_manual_testing = data_true_manual_testing.copy()
data_true_manual_testing['class'] = 1

# Merging manual test data

In [21]:
data_merge = pd.concat([data_fake_manual_testing, data_true_manual_testing], axis=0)
data_merge.head(10)
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


##  Combining Fake and True Data, Dropping Unnecessary Columns, and Shuffling

In [23]:
data = pd.concat([data_fake, data_true], axis=0)
data = data.drop(['title', 'subject', 'date'], axis=1)
data = data.sample(frac=1)
data.head()
data.columns

Index(['text', 'class'], dtype='object')

## Text Preprocessing Function for Cleaning News Content

In [25]:
def wordopt(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


In [27]:
data['text'] = data['text'].apply(wordopt)

# splitting the data

In [29]:
x = data['text']
y = data['class']

# vectorizing the text

In [31]:
vectorization = TfidfVectorizer()
xv = vectorization.fit_transform(x)

In [35]:
xv_train, xv_test, y_train, y_test = train_test_split(xv, y, test_size=0.25, random_state=42)

#  Logistic Regression Model

In [37]:
lr_model = LogisticRegression()
lr_model.fit(xv_train, y_train)
pred_lr = lr_model.predict(xv_test)

In [39]:
print("Accuracy (Logistic Regression):", accuracy_score(y_test, pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_lr))
print("Classification Report:\n", classification_report(y_test, pred_lr, zero_division=0))

Accuracy (Logistic Regression): 0.9869875222816399
Confusion Matrix:
 [[5829   82]
 [  64 5245]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      5911
           1       0.98      0.99      0.99      5309

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



#  Random Forest Model

In [44]:
rf_model = RandomForestClassifier()

In [45]:
rf_model.fit(xv_train, y_train)

RandomForestClassifier()

In [46]:
pred_rf = rf_model.predict(xv_test)

In [50]:
print("Accuracy (Random Forest):", accuracy_score(y_test, pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_rf))
print("Classification Report:\n", classification_report(y_test, pred_rf, zero_division=0))

Accuracy (Random Forest): 0.9892156862745098
Confusion Matrix:
 [[5865   46]
 [  75 5234]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      5911
           1       0.99      0.99      0.99      5309

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# Gradient Boosting Model

In [52]:
gb_model = GradientBoostingClassifier()
gb_model.fit(xv_train, y_train)

GradientBoostingClassifier()

In [55]:
pred_gb = gb_model.predict(xv_test)

In [57]:
print("Accuracy (Gradient Boosting):", accuracy_score(y_test, pred_gb))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_gb))
print("Classification Report:\n", classification_report(y_test, pred_gb, zero_division=0))

Accuracy (Gradient Boosting): 0.9958110516934047
Confusion Matrix:
 [[5875   36]
 [  11 5298]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      5911
           1       0.99      1.00      1.00      5309

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



# Manual Testing Function

In [59]:
def output_label(n):
    return "Fake" if n == 0 else "Real"

def manual_testing(news):
    news = wordopt(news)
    vectorized_input = vectorization.transform([news])

    pred_lr = lr_model.predict(vectorized_input)
    pred_rf = rf_model.predict(vectorized_input)
    pred_gb = gb_model.predict(vectorized_input)

    print("Logistic Regression Prediction:", output_label(pred_lr[0]))
    print("Random Forest Prediction:", output_label(pred_rf[0]))
    print("Gradient Boosting Prediction:", output_label(pred_gb[0]))

# Example Manual Test

In [63]:
manual_testing("Scientists discover a secret alien base on the dark side of the moon")

Logistic Regression Prediction: Fake
Random Forest Prediction: Fake
Gradient Boosting Prediction: Fake
